In [1]:
import numpy as np

def hes_transform(a):
    dim = a.shape[0]
    h = np.eye(dim)
    for i in range(0, dim-2):
        s = np.sign(a[i + 1, i]) * np.linalg.norm(a[i + 1:dim, i])
        mu = 1/np.sqrt(2 * s * (s - a[i + 1, i]))
        v = [0]
        for j in range(1, dim):
            if j < i+1:
                v.append(0)
            elif j == i+1:
                v.append(a[i + 1, i] - s)
            else:
                v.append(a[j, i])
        v = mu*np.matrix(v).transpose()
        h_ = (np.eye(dim) - 2*v*v.transpose())
        a = h_*a*h_
        h = h_*h
    return a, h

def givens_rotation(a):
    rows_, cols_ = np.shape(a)
    q = np.identity(rows_)
    r = np.copy(a)
    rows, cols = np.tril_indices(rows_, -1, cols_)
    for (row, col) in zip(rows, cols):
        if r[row, col] != 0:
            r1 = np.hypot(r[col, col], r[row, col])
            cos = r[col, col]/r1
            sin = -r[row, col]/r1
            g = np.identity(rows_)
            g[[col, row], [col, row]] = cos
            g[row, col] = sin
            g[col, row] = -sin
            r = np.dot(g, r)
            q = np.dot(q, g.T)
    return q, r

def qr_method(a, max_iter):
    #1
    a, h = hes_transform(a)

    for i in range(1, max_iter):
        #2
        q, r = givens_rotation(a)
        a = np.dot(r, q)

    return a

In [2]:
n = np.random.randint(3, 5)
lambdas = np.random.rand(n)*n*100
if lambdas[n-1]*lambdas[n-2] > 0:
    lambdas[n-1] *= -1
Alpha = np.diag(lambdas)
Alpha[[n-1, n-2]] = Alpha[[n-2, n-1]]
print("Altered Alpha:")
print(Alpha)
C = np.matrix(np.random.randn(n, n))*n
A = np.linalg.inv(C)*Alpha*C

Altered Alpha:
[[  79.18316774    0.            0.            0.        ]
 [   0.          128.59953563    0.            0.        ]
 [   0.            0.            0.         -385.4344516 ]
 [   0.            0.           61.22581442    0.        ]]


In [5]:
res = qr_method(A, 100)
print("Result matrix:")
print(res, "\n")

Result matrix:
[[ 2.08605399e+01 -2.98312212e+03 -2.93144677e+03 -3.60006655e+02]
 [ 8.05655935e+00 -2.08605242e+01 -1.59633107e+02  6.49435587e+01]
 [-2.25044844e-22 -1.54821817e-05  1.28599520e+02 -7.28409571e+01]
 [ 2.25634340e-43 -2.89990462e-41  7.06426690e-20  7.91831677e+01]] 



In [6]:
cell = res[0:-2, 0:-2]
print("Cell iterations:")
print(qr_method(cell, 1000))

Cell iterations:
[[ -984.13584225 -2611.23514999]
 [  379.94353368   984.13585799]]
